In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statistics
import pandas as pd

km_cost = 384
period_op_cost = 5400
dump_ind = 54
depot_ind = 56

skips_list = pd.read_csv('../../data/interm_data/skips_indexing.csv')

dump_arrival = pd.read_csv('../../data/interm_data/filtered_skip_deliveries_at_Mzedi.csv')
dump_arrival = dump_arrival.drop(columns=['end','deviceid', 'start','emptier type']).sort_values('date')
dump_arrival

distances = pd.read_csv('../../data/interm_data/dist_matrix_ID_filtered.csv')

#List of origin points
max_min = 1 #1 if max, 0 if min
list_origin = dump_arrival.skip_origin.unique().astype(str)

In [271]:
n = 0
tot_dist = 0
for i in dump_arrival.index:
    current_IDs = skips_list[skips_list.Name == dump_arrival['skip_origin'][i]].bin_ID.to_list()
    if len(current_IDs) != 0:
        current_dist = sum(distances.loc[((distances['FROM_ID'].isin(current_IDs)) & (distances['TO_ID'] == dump_ind)) | ((distances['TO_ID'].isin(current_IDs)) & (distances['FROM_ID'] == dump_ind))].DIST_KM.to_list())/len(current_IDs)
        dump_arrival.at[i,'mean_dist_trav'] = current_dist
        n = n+1
        tot_dist = tot_dist + current_dist
    
dump_arrival.mean_dist_trav[dump_arrival.mean_dist_trav.isna()] = tot_dist/n     

C:\Users\Nicolas Seemann\AppData\Local\Temp\ipykernel_18240\1588882571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dump_arrival.mean_dist_trav[dump_arrival.mean_dist_trav.isna()] = tot_dist/n


In [272]:
dump_back_forth_dist = sum(distances.loc[((distances['FROM_ID']== depot_ind) & (distances['TO_ID'] == dump_ind)) | ((distances['FROM_ID']== dump_ind) & (distances['TO_ID'] == depot_ind))].DIST_KM.to_list())

In [281]:
dump_back_forth_dist

21.25087

In [284]:
dump_arrival.date = pd.to_datetime(dump_arrival.date)
dump_depot_week_costs = ((dump_arrival.set_index('date')['mean_dist_trav'].resample('D').sum() > 0)*1).resample('W-MON').sum()*km_cost*2*dump_back_forth_dist
costs_week = dump_arrival.set_index('date')['mean_dist_trav'].resample('W-MON').sum()*km_cost + dump_depot_week_costs
costs_week_nnz = costs_week[costs_week != 0]
print(statistics.mean(costs_week)+14*period_op_cost)
print(statistics.mean(costs_week_nnz)+14*period_op_cost)
print(14*period_op_cost)
print(costs_week)

398169.17727520026
422514.0208431399
75600
date
2020-12-07    252632.321379
2020-12-14    300742.341583
2020-12-21    391958.172535
2020-12-28    265178.625379
2021-01-04     88547.573760
2021-01-11    383414.678312
2021-01-18    393081.071183
2021-01-25    127156.718632
2021-02-01         0.000000
2021-02-08         0.000000
2021-02-15         0.000000
2021-02-22         0.000000
2021-03-01    305243.945423
2021-03-08    231450.862996
2021-03-15    397607.680739
2021-03-22    373477.764303
2021-03-29    253764.399272
2021-04-05    289416.093263
2021-04-12    126393.874472
2021-04-19    147843.291708
2021-04-26    358752.954715
2021-05-03    311338.417832
2021-05-10    375128.202891
2021-05-17    196334.314251
2021-05-24    351329.426571
2021-05-31    155573.806908
2021-06-07    389473.745291
2021-06-14    429793.721810
2021-06-21    212309.116811
2021-06-28    220770.445087
2021-07-05    409703.641523
2021-07-12    328778.591647
2021-07-19    591515.941319
2021-07-26    514153.902179
